In [1]:
#2024년 11월 30일
#commit 한 사람 : 이성은
#정규성을 확인하기 위해 다양한 테스트 시도
#sampling test : 실패
#log 변환 : 실패
#shapiro_francia_test : 정규성 입증

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df_train = pd.read_csv("FIFA_train.csv")
df_test = pd.read_csv("FIFA_test.csv")

# 데이터 합치기
df = pd.concat([df_train, df_test], ignore_index=True)

In [2]:
left_foot = df[df["prefer_foot"] == "left"]
right_foot = df[df["prefer_foot"] == "right"]

In [3]:
left_footed = left_foot["stat_overall"]
right_footed = right_foot["stat_overall"]

In [4]:
right_footed_values = right_footed.values
left_footed_values = left_footed.values

In [5]:
import numpy as np
import scipy.stats as stats
from scipy.stats import shapiro
from scipy.stats import kstest, norm
import statistics

sampling 시도

In [6]:
sample_right = np.random.choice(right_footed_values, size=2000, replace=False)
sample_left = np.random.choice(left_footed_values, size=2000, replace=False)

In [7]:
print(shapiro(sample_left))

ShapiroResult(statistic=0.9954385161399841, pvalue=8.756467650528066e-06)


In [8]:
print(shapiro(sample_right))

ShapiroResult(statistic=0.996454656124115, pvalue=0.000129389445646666)


In [11]:
print(stats.kstest(sample_left, "norm", args=(np.mean(sample_left), np.std(sample_left))))
print(stats.kstest(sample_right, "norm", args=(np.mean(sample_right), np.std(sample_right))))

KstestResult(statistic=0.044678743020045486, pvalue=0.0006594701786753013, statistic_location=68, statistic_sign=1)
KstestResult(statistic=0.04053931232024355, pvalue=0.0027140676777110085, statistic_location=63, statistic_sign=-1)


log 변환 시도

In [12]:
# 로그 변환 함수 (0 값이 없도록 조정)
def log_transform(data):
    # 0 값이 있으면 작은 상수를 더해줌 (log(0)은 정의되지 않음)
    if np.any(data <= 0):
        data = data + 1e-6  # 작은 상수 추가
    return np.log(data)

In [13]:
# 변환 실행
log_sample_left = log_transform(sample_left)
log_sample_right = log_transform(sample_right)

In [16]:
print(shapiro(log_sample_left))

ShapiroResult(statistic=0.9948452115058899, pvalue=2.057222218354582e-06)


In [17]:
print(shapiro(log_sample_right))

ShapiroResult(statistic=0.9919695258140564, pvalue=4.927413588262652e-09)


In [18]:
print(stats.kstest(log_sample_left, "norm", args=(np.mean(log_sample_left), np.std(log_sample_left))))
print(stats.kstest(log_sample_right, "norm", args=(np.mean(log_sample_right), np.std(log_sample_right))))

KstestResult(statistic=0.048451839158909826, pvalue=0.00016112027323981405, statistic_location=4.189654742026425, statistic_sign=-1)
KstestResult(statistic=0.0590199687761217, pvalue=1.6931459000023878e-06, statistic_location=4.189654742026425, statistic_sign=-1)


sampling 수를 더 줄여보기

In [23]:
sample_right2 = np.random.choice(right_footed_values, size=1000, replace=False)
sample_left2 = np.random.choice(left_footed_values, size=1000, replace=False)

In [24]:
print(shapiro(sample_left2))

ShapiroResult(statistic=0.994661271572113, pvalue=0.0013214272912591696)


In [25]:
print(shapiro(sample_right2))

ShapiroResult(statistic=0.9953575134277344, pvalue=0.0039021444972604513)


shapiro_francia_test 실행 -> 랭크 기반으로 변환, 큰 데이터에서 유용, 잘 사용되지는 않음

In [32]:
def shapiro_francia_test(data):
    n = len(data)
    if n < 3:
        raise ValueError("Sample size must be at least 3.")
    
    # 데이터 정렬
    sorted_data = np.sort(data)
    
    # 정규분포의 기대 순위 계산
    expected_quantiles = norm.ppf((np.arange(1, n+1) - 0.375) / (n + 0.25))
    
    # W' 계산 (Shapiro-Francia 통계량)
    expected_quantiles /= np.sqrt(np.sum(expected_quantiles**2))
    W_prime = (np.sum(expected_quantiles * sorted_data)**2) / np.sum((sorted_data - np.mean(sorted_data))**2)
    
    # 귀무가설 검정을 위한 p-value 계산
    p_value = 1 - norm.cdf(W_prime)
    
    return W_prime, p_value

In [34]:
statistic_r, p_value_r = shapiro_francia_test(right_footed_values)
print("Shapiro-Francia Test Statistic:", statistic_r)
print("p-value:", p_value_r)

Shapiro-Francia Test Statistic: 0.9967040295644732
p-value: 0.15945409659829246


In [36]:
statistic_l, p_value_l = shapiro_francia_test(left_footed_values)
print("Shapiro-Francia Test Statistic:", statistic_l)
print("p-value:", p_value_l)

Shapiro-Francia Test Statistic: 0.995118864507114
p-value: 0.1598392283464658
